In [2]:
import pandas as pd
import numpy as np
import re

## Cargar datasets

In [4]:
# Box1 database
box = pd.read_csv('box1_proteins.csv')
box.head()

,uniprot,organism,mlo,hgnc_id,length,sequence
0,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...
1,O43663,Homo sapiens,pcg body,HGNC:9341,620,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...
2,O75494,Homo sapiens,nuclear speckle,HGNC:16713,262,MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFY...
3,O95613,Homo sapiens,centrosome/spindle pole body,HGNC:16068,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...
4,P08047,Homo sapiens,centrosome/spindle pole body,HGNC:11205,785,MSDQDHSMDEMTAVVKIEKGVGGNNGGNGNGGGAFSQARSSSTGSS...


In [11]:
# de Ensembl Biomart genere una tabla con gene_id y los posiblos transcriptos para cada uniprot de box1
bm = pd.read_csv('mart_export_uniprotid.tsv.gz', sep='\t', compression='gzip')
bm.columns = bm.columns.str.lower().str.replace(' ',"_").str.replace("-",'_').str.replace('/','_')
bm = bm.rename(columns={'chromosome_scaffold_name' : 'chromosome', 'uniprotkb_swiss_prot_id': 'uniprot', 'uniprotkb_isoform_id':'uniprot_isoform'})
bm

,gene_stable_id,transcript_stable_id,gene_name,source_of_gene_name,transcript_count,hgnc_id,uniprot_isoform,uniprot,chromosome,gene_start_(bp),gene_end_(bp)
0,ENSG00000204272,ENST00000637096,NBDY,HGNC Symbol,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
1,ENSG00000204272,ENST00000374922,NBDY,HGNC Symbol,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
2,ENSG00000204272,ENST00000423617,NBDY,HGNC Symbol,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
3,ENSG00000254726,ENST00000532414,MEX3A,HGNC Symbol,2,HGNC:33482,NaN,A1L020,1,156072013,156082465
4,ENSG00000178199,ENST00000409806,ZC3H12D,HGNC Symbol,5,HGNC:21175,A2A288-1,A2A288,6,149446795,149485014
...,...,...,...,...,...,...,...,...,...,...,...
1745,ENSG00000198492,ENST00000542507,YTHDF2,HGNC Symbol,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775
1746,ENSG00000198492,ENST00000373812,YTHDF2,HGNC Symbol,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775
1747,ENSG00000198492,ENST00000541996,YTHDF2,HGNC Symbol,9,HGNC:31675,Q9Y5A9-2,Q9Y5A9,1,28736621,28769775
1748,ENSG00000106723,ENST00000375859,SPIN1,HGNC Symbol,6,HGNC:11243,NaN,Q9Y657,9,88388430,88478694


In [12]:
bm['chr'] = bm.chromosome.map(lambda x: re.findall('[A-W].*', x)) # busco todos los que no son chr reales
bm['chr'] = bm.chr.str[0]
bm = bm[bm.chr.isnull()] # los que son nulos en esta col significa que son los reales en chromosome (X, Y, 6, etc)
bm = bm.drop(columns= ['chr', 'source_of_gene_name'])
bm

,gene_stable_id,transcript_stable_id,gene_name,transcript_count,hgnc_id,uniprot_isoform,uniprot,chromosome,gene_start_(bp),gene_end_(bp)
0,ENSG00000204272,ENST00000637096,NBDY,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
1,ENSG00000204272,ENST00000374922,NBDY,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
2,ENSG00000204272,ENST00000423617,NBDY,4,HGNC:50713,NaN,A0A0U1RRE5,X,56729241,56819179
3,ENSG00000254726,ENST00000532414,MEX3A,2,HGNC:33482,NaN,A1L020,1,156072013,156082465
4,ENSG00000178199,ENST00000409806,ZC3H12D,5,HGNC:21175,A2A288-1,A2A288,6,149446795,149485014
...,...,...,...,...,...,...,...,...,...,...
1745,ENSG00000198492,ENST00000542507,YTHDF2,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775
1746,ENSG00000198492,ENST00000373812,YTHDF2,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775
1747,ENSG00000198492,ENST00000541996,YTHDF2,9,HGNC:31675,Q9Y5A9-2,Q9Y5A9,1,28736621,28769775
1748,ENSG00000106723,ENST00000375859,SPIN1,6,HGNC:11243,NaN,Q9Y657,9,88388430,88478694


In [13]:
# creo una serie con las isoformas
isoformas = pd.Series((bm[bm.uniprot_isoform.notnull()]).uniprot_isoform.unique()) # no todas las prot tienen isoformas
#isoformas.to_csv('isoformas.csv', sep=',', index=False, header=None)

In [28]:
nn = bm[bm.uniprot.notnull() & bm.uniprot_isoform.notnull()]

In [29]:
nn

,gene_stable_id,transcript_stable_id,gene_name,transcript_count,hgnc_id,uniprot_isoform,uniprot,chromosome,gene_start_(bp),gene_end_(bp)
4,ENSG00000178199,ENST00000409806,ZC3H12D,5,HGNC:21175,A2A288-1,A2A288,6,149446795,149485014
5,ENSG00000105793,ENST00000257659,GTPBP10,12,HGNC:25106,A4D1E9-2,A4D1E9,7,90335223,90391453
6,ENSG00000105793,ENST00000380058,GTPBP10,12,HGNC:25106,A4D1E9-3,A4D1E9,7,90335223,90391453
7,ENSG00000105793,ENST00000222511,GTPBP10,12,HGNC:25106,A4D1E9-1,A4D1E9,7,90335223,90391453
8,ENSG00000125107,ENST00000317147,CNOT1,23,HGNC:7877,A5YKK6-1,A5YKK6,16,58519951,58629885
...,...,...,...,...,...,...,...,...,...,...
1741,ENSG00000106355,ENST00000409782,LSM5,10,HGNC:17162,Q9Y4Y9-2,Q9Y4Y9,7,32485338,32495283
1742,ENSG00000106355,ENST00000409952,LSM5,10,HGNC:17162,Q9Y4Y9-2,Q9Y4Y9,7,32485338,32495283
1745,ENSG00000198492,ENST00000542507,YTHDF2,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775
1746,ENSG00000198492,ENST00000373812,YTHDF2,9,HGNC:31675,Q9Y5A9-1,Q9Y5A9,1,28736621,28769775


In [25]:
len(bm.uniprot.unique())

582

In [30]:
len(nn.uniprot.unique())

388

In [32]:
m = bm[['uniprot', 'uniprot_isoform']]

In [33]:
len(m.uniprot.unique())


582

In [39]:
mm = m.drop_duplicates()
mm

,uniprot,uniprot_isoform
0,A0A0U1RRE5,NaN
3,A1L020,NaN
4,A2A288,A2A288-1
5,A4D1E9,A4D1E9-2
6,A4D1E9,A4D1E9-3
...,...,...
1744,Q9Y580,NaN
1745,Q9Y5A9,Q9Y5A9-1
1747,Q9Y5A9,Q9Y5A9-2
1748,Q9Y657,NaN


In [40]:
len(mm.uniprot.unique())

582

# DisGeNET  
## variant-disease association dataset

In [42]:
# variant-disease association
# este dataset viene de la pagina de downloads
vda = pd.read_csv('curated_variant_disease_associations.tsv', sep='\t')
vda.columns = vda.columns.str.lower().str.replace(' ',"_").str.replace("-",'_').str.replace('/','_')
vda = vda.rename(columns={'snpid':'snp_id'})

In [43]:
# Summary of Curated VDAs
# este dataset viene de buscar en la web de disgenet y descargarlo completo
# lo uso xq tiene mas info que complementa al de arriba
sumvda = pd.read_csv('summaryofvdas_CURATED.tsv', sep='\t')
sumvda.columns = sumvda.columns.str.lower().str.replace(' ',"_").str.replace("-",'_').str.replace('/','_')
sumvda = sumvda.rename(columns={'variant':'snp_id', 'chr':'chromosome', 'gene': 'gene_name'})
sumvda = sumvda[['snp_id', 'gene_name', 'gene_id', 'consequence', 'alleles', 'class',
        'af_exome', 'alt_ref_exome', 'af_genome', 'alt_ref_genome']]

In [45]:
vda

,snp_id,chromosome,position,dsi,dpi,diseaseid,diseasename,diseasetype,diseaseclass,diseasesemantictype,score,ei,yearinitial,yearfinal,nofpmids,source
0,rs1000005,21,33060745,NaN,NaN,C0200638,Eosinophil count procedure,phenotype,NaN,Laboratory Procedure,0.7,1.0,2016.0,2016.0,1,GWASCAT
1,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,0.7,1.0,2017.0,2017.0,1,GWASCAT
2,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,0.7,1.0,2007.0,2007.0,1,CLINVAR
3,rs1000096,4,38691214,NaN,NaN,C1305855,Body mass index,phenotype,NaN,Clinical Attribute,0.7,1.0,2019.0,2019.0,1,GWASCAT
4,rs10001106,4,10125817,0.925,0.12,C0003868,"Arthritis, Gouty",disease,C16;C18;C05,Disease or Syndrome,0.7,1.0,2010.0,2013.0,3,GWASDB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261222,rs9999118,4,123130312,1.000,0.04,C0022104,Irritable Bowel Syndrome,disease,C06,Disease or Syndrome,0.8,1.0,2014.0,2014.0,1,GWASCAT;GWASDB
261223,rs999921351,8,86628939,1.000,0.12,C1849792,Achromatopsia 3,disease,C23;C11;C10,Disease or Syndrome,0.7,1.0,2017.0,2017.0,1,CLINVAR
261224,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,0.8,1.0,2009.0,2009.0,1,GWASCAT;GWASDB
261225,rs999944,2,64822719,1.000,0.08,C0520679,"Sleep Apnea, Obstructive",disease,C08;C10,Disease or Syndrome,0.7,1.0,2016.0,2016.0,1,GWASCAT


In [44]:
sumvda

,snp_id,gene_name,gene_id,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome
0,rs1000005,LINC00945,101930746,intron variant,G/C,snv,NaN,NaN,0.505584,G/C
1,rs10000770,INPP4B,8821,intron variant,C/T,snv,NaN,NaN,0.108425,C/T
2,rs1000091588,IGHMBP2,3508,stop gained,C/T,snv,NaN,NaN,0.000007,C/T
3,rs1000096,KLF3,51274,intron variant,C/T,snv,NaN,NaN,0.387045,C/T
4,rs10001106,NaN,NaN,intergenic variant,T/A;C,snv,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
261222,rs9999118,SPATA5,166378,intron variant,A/G,snv,NaN,NaN,0.012246,A/G
261223,rs999921351,CNGB3,54714,stop gained,C/T,snv,NaN,NaN,NaN,NaN
261224,rs999943,ITPR3,3710,intron variant,A/G,snv,NaN,NaN,0.282840,A/G
261225,rs999944,NaN,NaN,intergenic variant,A/G,snv,NaN,NaN,0.833959,A/G


In [111]:
# uno ambas tablas, axis=1 es por columnas
disgenet_vdas = pd.concat([vda, sumvda], axis=1)
# Elimino la col duplicada (snp_id, son identicas)
disgenet_vdas = disgenet_vdas.loc[:, ~disgenet_vdas.columns.duplicated()]
disgenet_vdas

,snp_id,chromosome,position,dsi,dpi,diseaseid,diseasename,diseasetype,diseaseclass,diseasesemantictype,...,source,gene_name,gene_id,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome
0,rs1000005,21,33060745,NaN,NaN,C0200638,Eosinophil count procedure,phenotype,NaN,Laboratory Procedure,...,GWASCAT,LINC00945,101930746,intron variant,G/C,snv,NaN,NaN,0.505584,G/C
1,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,...,GWASCAT,INPP4B,8821,intron variant,C/T,snv,NaN,NaN,0.108425,C/T
2,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,...,CLINVAR,IGHMBP2,3508,stop gained,C/T,snv,NaN,NaN,0.000007,C/T
3,rs1000096,4,38691214,NaN,NaN,C1305855,Body mass index,phenotype,NaN,Clinical Attribute,...,GWASCAT,KLF3,51274,intron variant,C/T,snv,NaN,NaN,0.387045,C/T
4,rs10001106,4,10125817,0.925,0.12,C0003868,"Arthritis, Gouty",disease,C16;C18;C05,Disease or Syndrome,...,GWASDB,NaN,NaN,intergenic variant,T/A;C,snv,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261222,rs9999118,4,123130312,1.000,0.04,C0022104,Irritable Bowel Syndrome,disease,C06,Disease or Syndrome,...,GWASCAT;GWASDB,SPATA5,166378,intron variant,A/G,snv,NaN,NaN,0.012246,A/G
261223,rs999921351,8,86628939,1.000,0.12,C1849792,Achromatopsia 3,disease,C23;C11;C10,Disease or Syndrome,...,CLINVAR,CNGB3,54714,stop gained,C/T,snv,NaN,NaN,NaN,NaN
261224,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,...,GWASCAT;GWASDB,ITPR3,3710,intron variant,A/G,snv,NaN,NaN,0.282840,A/G
261225,rs999944,2,64822719,1.000,0.08,C0520679,"Sleep Apnea, Obstructive",disease,C08;C10,Disease or Syndrome,...,GWASCAT,NaN,NaN,intergenic variant,A/G,snv,NaN,NaN,0.833959,A/G


In [112]:
# hay nans
disgenet_vdas.gene_id.isnull().value_counts()

False    225675
True      35552
Name: gene_id, dtype: int64

In [115]:
# la col gene_id debe ser tipo integer
disgenet_vdas.gene_id = disgenet_vdas.gene_id.astype(np.int64, errors='ignore')

In [118]:
disgenet_vdas.gene_id[9995]

'91319;6598'

In [211]:
disgenet_vdas.nofpmids.value_counts()

1     137361
0      78085
2      15806
3       6570
4       3887
5       2775
20      2598
6       2090
7       1666
8       1273
9        974
10       904
12       852
21       736
11       705
15       614
13       577
14       518
16       411
18       383
22       350
17       348
19       307
24       180
26       165
23       164
25       159
31       110
34       100
27        83
30        78
28        77
29        62
32        45
35        32
36        27
33        26
37        25
41        15
38        14
39        14
40        12
46        11
44         5
43         5
42         5
45         4
47         4
48         3
51         3
52         3
50         2
49         1
53         1
56         1
76         1
Name: nofpmids, dtype: int64

In [50]:
disgenet_vdas['class'].value_counts()

snv       222832
delins     29290
del         7457
ins         1175
mnv          473
Name: class, dtype: int64

In [51]:
disgenet_vdas['source'].value_counts()

CLINVAR                   106710
GWASCAT                    83276
GWASDB                     41771
UNIPROT                    12032
CLINVAR;UNIPROT            11259
GWASCAT;GWASDB              6164
CLINVAR;GWASCAT                7
GWASCAT;GWASDB;UNIPROT         4
CLINVAR;GWASCAT;GWASDB         2
GWASCAT;UNIPROT                1
CLINVAR;GWASDB                 1
Name: source, dtype: int64

### Uniprot gene id cross-reference

In [119]:
# Mappings UniProts: uniprots id con entrez gene id (https://www.disgenet.org/downloads, Mappings)
uniprots = pd.read_csv('mapa_geneid_4_uniprot_crossref.tsv.gz', sep='\t', compression='gzip')
uniprots = uniprots.rename(columns={'UniProtKB':'uniprot', 'GENEID':'gene_id'})

### Variant to Gene mappings

In [126]:
# The file contains the mappings of DisGeNET variants (dbSNP Identifiers)
# to NCBI Entrez identifiers according to dbSNP database (https://www.disgenet.org/downloads, Variant-Gene Mappings File)
variant_gene = pd.read_csv('variant_to_gene_mappings.tsv.gz', sep='\t', compression='gzip')
variant_gene = variant_gene.rename(columns={'snpId':'snp_id', 'geneId':'gene_id', 'geneSymbol': 'gene_name', 'sourceId':'source_id'})


In [127]:
# agrego los uniprots
variant_gene = variant_gene.merge(uniprots)

In [136]:
disgenet_total = disgenet_vdas.merge(variant_gene, on=['snp_id', 'gene_name'], how='left')
disgenet_total

,snp_id,chromosome,position,dsi,dpi,diseaseid,diseasename,diseasetype,diseaseclass,diseasesemantictype,...,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome,gene_id_y,source_id,uniprot
0,rs1000005,21,33060745,NaN,NaN,C0200638,Eosinophil count procedure,phenotype,NaN,Laboratory Procedure,...,intron variant,G/C,snv,NaN,NaN,0.505584,G/C,NaN,NaN,NaN
1,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,...,intron variant,C/T,snv,NaN,NaN,0.108425,C/T,8821.0,DBSNP,O15327
2,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,...,intron variant,C/T,snv,NaN,NaN,0.108425,C/T,8821.0,VEP,O15327
3,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,...,stop gained,C/T,snv,NaN,NaN,0.000007,C/T,3508.0,VEP,P38935
4,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,...,stop gained,C/T,snv,NaN,NaN,0.000007,C/T,3508.0,DBSNP,P38935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433635,rs999921351,8,86628939,1.000,0.12,C1849792,Achromatopsia 3,disease,C23;C11;C10,Disease or Syndrome,...,stop gained,C/T,snv,NaN,NaN,NaN,NaN,54714.0,VEP,Q9NQW8
433636,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,...,intron variant,A/G,snv,NaN,NaN,0.282840,A/G,3710.0,VEP,Q14573
433637,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,...,intron variant,A/G,snv,NaN,NaN,0.282840,A/G,3710.0,DBSNP,Q14573
433638,rs999944,2,64822719,1.000,0.08,C0520679,"Sleep Apnea, Obstructive",disease,C08;C10,Disease or Syndrome,...,intergenic variant,A/G,snv,NaN,NaN,0.833959,A/G,NaN,NaN,NaN


## Merge box1 con disgenet total

In [191]:
#merge de box con los datos totales de vda de disgenet
box_disgenet = box.merge(disgenet_total, on='uniprot', how='left')

In [192]:
len(box_disgenet.uniprot.unique())

589

In [193]:
# hay 245 entradas sin snp
box_disgenet.snp_id.isnull().value_counts()

False    22743
True       245
Name: snp_id, dtype: int64

In [194]:
# elimino las entradas sin snp
box_disgenet = box_disgenet.dropna(subset= ['snp_id'])

In [195]:
len(box_disgenet.uniprot.unique())

344

In [196]:
box_disgenet.gene_id_x = box_disgenet.gene_id_x.apply(int)
box_disgenet.gene_id_y = box_disgenet.gene_id_y.apply(int)

In [197]:
box_disgenet = box_disgenet.drop_duplicates(subset=['snp_id', 'diseaseid'], keep='first')
# hay entradas duplicadas xq source_id puede venir de DBSNP o de VEP

In [198]:
len(box_disgenet)

11549

In [199]:
len(box_disgenet.snp_id.unique())

5942

In [200]:
(box_disgenet.gene_id_x ==  box_disgenet.gene_id_y).value_counts() # son todos iguales, elimino una col

True    11549
dtype: int64

In [201]:
box_disgenet = box_disgenet.drop(columns=['gene_id_y'])
box_disgenet = box_disgenet.rename(columns={'gene_id_x': 'gene_id'})

In [202]:
box_disgenet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11549 entries, 0 to 22985
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   uniprot              11549 non-null  object 
 1   organism             11549 non-null  object 
 2   mlo                  11549 non-null  object 
 3   hgnc_id              11539 non-null  object 
 4   length               11549 non-null  int64  
 5   sequence             11549 non-null  object 
 6   snp_id               11549 non-null  object 
 7   chromosome           11549 non-null  object 
 8   position             11549 non-null  float64
 9   dsi                  9917 non-null   float64
 10  dpi                  9380 non-null   float64
 11  diseaseid            11549 non-null  object 
 12  diseasename          11549 non-null  object 
 13  diseasetype          11549 non-null  object 
 14  diseaseclass         8812 non-null   object 
 15  diseasesemantictype  11549 non-null 

In [203]:
box_disgenet

,uniprot,organism,mlo,hgnc_id,length,sequence,snp_id,chromosome,position,dsi,...,gene_name,gene_id,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome,source_id
0,O15527,Homo sapiens,nuclear speckle,HGNC:8125,345,MPARALLPRRMGHRTLASTPALWASIPCPRSELRLDLVLPSGQSFR...,rs104893751,3,9750423.0,0.882,...,OGG1,4968,missense variant,G/A;C,snv,0.0022139;4.00343e-06,G/A;G/C,NaN,NaN,DBSNP
3,O75494,Homo sapiens,nuclear speckle,HGNC:16713,262,MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFY...,rs35233638,1,23968097.0,NaN,...,SRSF10,10772,3 prime UTR variant,G/A,snv,NaN,NaN,0.459401,G/A,DBSNP
5,O75494,Homo sapiens,nuclear speckle,HGNC:16713,262,MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFY...,rs35233638,1,23968097.0,NaN,...,SRSF10,10772,3 prime UTR variant,G/A,snv,NaN,NaN,0.459401,G/A,DBSNP
7,O95613,Homo sapiens,centrosome/spindle pole body,HGNC:16068,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...,rs119479061,21,46346146.0,1.000,...,PCNT,5116,stop gained,G/T,snv,NaN,NaN,NaN,NaN,DBSNP
9,O95613,Homo sapiens,centrosome/spindle pole body,HGNC:16068,3336,MEVEQEQRRRKVEAGRTKLAHFRQRKTKGDSSHSEKKTAKRKGSAV...,rs119479062,21,46411840.0,1.000,...,PCNT,5116,stop gained,C/T,snv,1.22206e-05,C/T,0.000014,C/T,VEP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,P26368,Homo sapiens,nuclear speckle,HGNC:23156,475,MSDFDEFERQLNENKQERDKENRHRKRSHSRSRSRDRKRRSRSRDR...,rs79546472,19,55662011.0,NaN,...,U2AF2,11338,non coding transcript exon variant,G/A;C,snv,NaN,NaN,NaN,NaN,VEP
22979,Q15233,Homo sapiens,paraspeckle,HGNC:7871,471,MQSNKTFNLEKQNHTPRKHHQHHHQQQHHQQQQQQPPPPPIPANGQ...,rs1114167441,X,71298508.0,1.000,...,NONO,4841,missense variant,G/T,snv,NaN,NaN,NaN,NaN,VEP
22981,Q15233,Homo sapiens,paraspeckle,HGNC:7871,471,MQSNKTFNLEKQNHTPRKHHQHHHQQQHHQQQQQQPPPPPIPANGQ...,rs869025343,X,71297938.0,1.000,...,NONO,4841,splice region variant,G/A,snv,NaN,NaN,NaN,NaN,DBSNP
22983,Q15233,Homo sapiens,paraspeckle,HGNC:7871,471,MQSNKTFNLEKQNHTPRKHHQHHHQQQHHQQQQQQPPPPPIPANGQ...,rs869025345,X,71297900.0,1.000,...,NONO,4841,stop gained,C/T,snv,NaN,NaN,NaN,NaN,VEP


In [206]:
box_disgenet['class'].value_counts()

snv       9369
delins    1681
del        413
ins         48
mnv         38
Name: class, dtype: int64

In [207]:
box_disgenet.source.value_counts()

CLINVAR            7720
GWASCAT            1774
GWASDB              887
UNIPROT             513
CLINVAR;UNIPROT     498
GWASCAT;GWASDB      156
CLINVAR;GWASDB        1
Name: source, dtype: int64

In [208]:
box_disgenet[box_disgenet.source == 'CLINVAR;UNIPROT']

,uniprot,organism,mlo,hgnc_id,length,sequence,snp_id,chromosome,position,dsi,...,gene_name,gene_id,consequence,alleles,class,af_exome,alt_ref_exome,af_genome,alt_ref_genome,source_id
152,P42858,Homo sapiens,centrosome/spindle pole body,HGNC:4851,3142,MATLEKLMKAFESLKSFQQQQQQQQQQQQQQQQQQQQQPPPPPPPP...,rs768047421,4,3131647.0,1.000,...,HTT,3064,missense variant,C/T,snv,8.05529e-05,C/T,0.000035,C/T,DBSNP
162,P46100,Homo sapiens,pml nuclear body,HGNC:886,2492,MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...,rs1057518708,X,77688843.0,1.000,...,ATRX,546,missense variant,G/A,snv,NaN,NaN,NaN,NaN,DBSNP
176,P46100,Homo sapiens,pml nuclear body,HGNC:886,2492,MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...,rs122445093,X,77633696.0,1.000,...,ATRX,546,missense variant,T/C,snv,NaN,NaN,NaN,NaN,VEP
178,P46100,Homo sapiens,pml nuclear body,HGNC:886,2492,MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...,rs122445094,X,77633682.0,1.000,...,ATRX,546,missense variant,A/G,snv,NaN,NaN,NaN,NaN,DBSNP
180,P46100,Homo sapiens,pml nuclear body,HGNC:886,2492,MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...,rs122445095,X,77633572.0,1.000,...,ATRX,546,missense variant,C/A;T,snv,NaN,NaN,NaN,NaN,VEP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22812,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,rs74315408,20,4699758.0,0.752,...,PRNP,5621,missense variant,G/A,snv,6.36264e-05,G/A,0.000042,G/A,DBSNP
22820,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,rs74315410,20,4699612.0,1.000,...,PRNP,5621,missense variant,G/T,snv,3.97779e-06,G/T,NaN,NaN,DBSNP
22824,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,rs74315411,20,4699767.0,0.882,...,PRNP,5621,missense variant,A/G,snv,NaN,NaN,NaN,NaN,DBSNP
22830,P04156,Homo sapiens,cytoplasmic protein granule,HGNC:9449,253,MANLGCWMLVLFVATWSDLGLCKKRPKPGGWNTGGSRYPGQGSPGG...,rs74315413,20,4699780.0,0.807,...,PRNP,5621,missense variant,A/G,snv,NaN,NaN,NaN,NaN,DBSNP
